


{目标网页}：

- url: 'https://www.kaggle.com/datasets'
- data_name: 'adilshamim8/credit-risk-benchmark-dataset'
- type: 'data'
- 完整网址：https://www.kaggle.com/datasets/adilshamim8/credit-risk-benchmark-dataset/data


==目的==：我想编写一个通用的 Python 函数，从 kaggle.com/datasets 网站的 {目标网页} 中提取数据集的主要信息，做一个精简版的数据集介绍。

我期望获取的主要内容包括：

- 数据集的名称：[xxx](URL)，[Download](URL)
- 数据集的描述：xxx 页面中 [About Dataset] 部分的所有内容。但需要采用比较精简的方式来呈现。
- 数据集的大小：N = ？; P = ？
- 数据集的类别：从网页中提取。(例如：分类，回归，聚类，时间序列)
- 数据集的来源：Kaggle，UCI，OpenML

Python 函数的名称：'kaggle_data'

函数的输入参数：
- url：数据集的 URL 地址

输出：
- 一个 .md 文档，或者 Markdown 格式的文本，让我可以直接复制到讲义中。
- .md 文档的名称：
  - 数据集名称.md，如 'credit-risk-benchmark-dataset.md'
- 语言：简体中文

In [ ]:
def kaggle_data(url):
    """
    从Kaggle数据集页面提取主要信息，并生成Markdown格式的介绍文本。
    参数:
        url: Kaggle数据集页面的URL
    返回:
        markdown_text: Markdown格式的介绍文本
        filename: 推荐的.md文件名
    """
    return extract_kaggle_dataset_info(url)

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def extract_kaggle_dataset_info(url):
    """
    从Kaggle数据集页面提取主要信息，并生成Markdown格式的介绍文本。
    参数:
        url: Kaggle数据集页面的URL
    返回:
        markdown_text: Markdown格式的介绍文本
        filename: 推荐的.md文件名
    """
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")

    # 数据集名称
    title_tag = soup.find("h1")
    dataset_name = title_tag.text.strip() if title_tag else "未知数据集"
    # 数据集描述
    about_section = soup.find("div", string=re.compile("About Dataset|数据集简介"))
    if about_section:
        desc = about_section.find_next("div").text.strip()
    else:
        desc = "暂无描述信息。"

    # 数据集大小（N, P）
    n, p = "未知", "未知"
    stats = soup.find_all("span", string=re.compile(r"\d+"))
    for s in stats:
        if "rows" in s.parent.text:
            n = re.sub(r"\D", "", s.text)
        if "columns" in s.parent.text:
            p = re.sub(r"\D", "", s.text)

    # 数据集类别（尝试从描述中猜测）
    category = "未知"
    if re.search("classification|分类", desc, re.I):
        category = "分类"
    elif re.search("regression|回归", desc, re.I):
        category = "回归"
    elif re.search("cluster|聚类", desc, re.I):
        category = "聚类"
    elif re.search("time series|时间序列", desc, re.I):
        category = "时间序列"

    # 来源
    source = "Kaggle"

    # 下载链接
    download_url = url.rstrip("/") + "/download"

    # Markdown文本
    markdown_text = f"""# [{dataset_name}]({url})  [Download]({download_url})

**数据集描述**：
{desc}

- 数据集大小：N = {n}；P = {p}
- 数据集类别：{category}
- 数据集来源：{source}
"""

    # 文件名
    dataset_slug = url.rstrip("/").split("/")[-1]
    filename = f"{dataset_slug}.md"
    return markdown_text, filename

# 示例用法
md_text, md_filename = extract_kaggle_dataset_info("https://www.kaggle.com/datasets/adilshamim8/credit-risk-benchmark-dataset")
print(md_text)